# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Image Classification Service with Gemini 2.5 Flash

This notebook demonstrates how to classify images from Imagery Insights using the Gemini 2.5 Flash model via Vertex AI.

## Install Required Libraries

In [ ]:
!pip install --upgrade google-cloud-bigquery google-cloud-aiplatform

## Configuration

**Important**: Replace the placeholder values below with your actual GCP Project ID and Region.

In [ ]:
PROJECT_ID = ''  # @param {type:"string"}
REGION = 'us-central1'      # @param {type:"string"}

## Imports and Vertex AI Initialization

In [ ]:
import vertexai
from google.cloud import bigquery
from vertexai.generative_models import GenerativeModel, Part

# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=REGION)

## Fetch Image URIs from BigQuery

Next, we'll query a BigQuery table to get the GCS URIs of the images we want to classify.

In [ ]:
BIGQUERY_SQL_QUERY = """
SELECT
  *
FROM
  `sarthaks-lab.imagery_insights___preview___us.latest_observations`
  WHERE asset_type = "ASSET_CLASS_UTILITY_POLE"

LIMIT 10;
"""

# Execute BigQuery Query
try:
    bigquery_client = bigquery.Client(project=PROJECT_ID)
    query_job = bigquery_client.query(BIGQUERY_SQL_QUERY)
    query_response_data = [dict(row) for row in query_job]
    gcs_uris = [item.get("gcs_uri") for item in query_response_data if item.get("gcs_uri")]

    print(f"Successfully fetched {len(gcs_uris)} GCS URIs:")
    for uri in gcs_uris:
        print(uri)
except Exception as e:
    print(f"An error occurred while querying BigQuery: {e}")

## Define Image Classification Function

This function takes a GCS URI and a prompt, then uses the Gemini 2.5 Flash model to generate a description of the image.

In [ ]:
def classify_image_with_gemini(gcs_uri: str, prompt: str) -> str:
    """
    Classifies an image using the Gemini 2.5 Flash model by directly passing its GCS URI.
    """
    MODEL = "gemini-2.5-flash" # @param {type:"string"}

    try:
        model = GenerativeModel(MODEL)
        image_part = Part.from_uri(uri=gcs_uri, mime_type="image/jpeg")
        responses = model.generate_content([image_part, prompt])
        return responses.text
    except Exception as e:
        print(f"Error classifying image from URI {gcs_uri}: {e}")
        return "Classification failed."

## Classify Images

Finally, we loop through the GCS URIs we fetched and pass them to our classification function along with a prompt.

In [ ]:
prompt = """You will be provided with a photo of a utility pole:
{photo_of_utility_pole}

Instructions:

1. Analyze the provided image. If the image does not clearly show a utility pole, return: {\"error\": \"No utility pole detected in the image.\"}
2. Detect and count the following:
    * Transformers
    * Power lines coming from the pole
    * Street lamps attached to the pole
    * Telephone or junction boxes
3. Assess the overall condition of the pole. Look for visible damage, bird nests, or other issues.  If the pole appears to be in good condition, note \"OK\".
4. Note the material with which the pole is made
5. Determine primary type of pole: Report this in the type field:
  * Street light
  * High tension power transmission
  * electricity pole
  * other
6. Provide your findings in the following JSON format:

```json
{
  \"pole_condition\": \"OK/Damaged/Other Issues\",
  \"type\": <pole_type>,
   \"material\":<material>
  \"transformers\": <number_of_transformers>,
  \"power_lines\": <number_of_power_lines>,
  \"street_lamps\": <number_of_street_lamps>,
  \"junction_boxes\": <number_of_junction_boxes>,
  \"additional_notes\": \"<any_other_observations>\"
}
```

Example:

Let's say the image shows a utility pole in good condition with one transformer, three power lines, one street lamp, and no junction boxes. The JSON output would be:

```json
{
  \"pole_condition\": \"OK\",
  \"type\": electricity pole,
  \"material\": wood,
  \"transformers\": 1,
  \"power_lines\": 3,
  \"street_lamps\": 1,
  \"junction_boxes\": 0,
  \"additional_notes\": \"None\"
}
"""

if 'gcs_uris' in locals() and gcs_uris:
    for uri in gcs_uris:
        print(f"--- Classifying {uri} ---")
        classification = classify_image_with_gemini(uri, prompt)
        print(f"Result: {classification}\n")
else:
    print("No GCS URIs were found to classify.")